In [321]:
import itertools
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import pyvis as pv
from pyvis.network import Network
import random

# NetworkX + Pyvis vizualization 

Generate simulation data. 

In [322]:
# generate a random data set with edges and their weights  
np.random.seed(888) # set a seed to make result reproducible 
df = pd.DataFrame(np.random.randint(low=0,high=250,size=(1000, 3)), columns=['source', 'target', 'weight'])
df.head()

,source,target,weight
0,154,237,239
1,150,115,103
2,46,249,188
3,109,145,144
4,221,12,96


In [323]:
# weight parameter is too large, so divide it by 10
df['weight'] = round(df['weight']/10)
df.head()

,source,target,weight
0,154,237,24.0
1,150,115,10.0
2,46,249,19.0
3,109,145,14.0
4,221,12,10.0


In [324]:
# check weights distribution
df.weight.value_counts()

4.0     58
22.0    53
8.0     52
6.0     50
10.0    46
15.0    44
9.0     41
5.0     40
24.0    40
20.0    40
11.0    40
12.0    39
2.0     39
14.0    39
3.0     38
23.0    38
16.0    36
13.0    35
1.0     35
21.0    35
7.0     33
18.0    33
17.0    31
19.0    31
0.0     18
25.0    16
Name: weight, dtype: int64

With data ready at our disposal, we can start with basic *pyvis*+*networkX* graph. By "basic" I mean plotting a *pyvis* network based on *networkX* node positioning algorithm; no additional manipulations with edges or nodes, no clustering, etc.

In [325]:
# get lists of nodes and connections(edges)
list_of_edges = df.values.tolist()
list_of_nodes = list(set(list(df.source) + list(df.target))) # need a set of unigue values from the entire dataset 

In [326]:
g = nx.Graph() # make an empty graph object with NetworkX
g.add_weighted_edges_from(list_of_edges) # add edges to the graph

In [327]:
# I use networkX spring layout (a node positioning algorithm) to calculate coordinates for nodes in my graph 
pos = nx.spring_layout(g, k=1, seed=555)
pos_df = (pd.DataFrame(pos).T)*2000 # scale coordinates by multiplying them with 2000
pos_x = list(pos_df[0]) # coords for X axis 
pos_y = list(pos_df[1]) # coords for Y axis

In [328]:
# having coordinates, I can set them in a pyvis network to visualize the graph 
net = Network("600px", "700px", notebook=True) 
# disable physics because our graph is pretty large and it would take too long for it to render + nodes will move too fast: 
net.toggle_physics(False) 
# similar way as above, add nodes and edges in the network 
net.add_nodes(list_of_nodes,
       x=pos_x,
       y=pos_y)
# instead of adding edges from the list_of_edges, I am adding them straight from networkX graph object:
net.add_edges(list(g.edges))
net.inherit_edge_colors(False) # set this option to False because I don't want edges to have the same color as nodes
net.set_edge_smooth('horizontal') # similarly, I want edges to have a nice smooth shape

In [329]:
net.show('network_graph.html')

Looks like there are too many edges and nodes. We can remove them straight from networkX graph object:

In [330]:
# remove all nodes that e.g. have weight smaller than 20
to_remove = [(a,b) for a, b, attrs in g.edges(data=True) if attrs["weight"] < 20]
g.remove_edges_from(to_remove)

In [331]:
# get rid of "lonely" nodes that were left with no edges:
isolated_nodes = list(nx.isolates(g))
g.remove_nodes_from(isolated_nodes)

In [332]:
# calculate new list of nodes for the pyvis graph
new_list_of_nodes = [j for j in list_of_nodes if j not in isolated_nodes]

In [333]:
# draw the new graph:
pos = nx.spring_layout(g, k=1, seed=555) # slightly adjusted edges lengt in the layout with argument k
pos_df = (pd.DataFrame(pos).T)*2000 # scale coordinates 
pos_x = list(pos_df[0]) # new coords for X axis 
pos_y = list(pos_df[1]) # new coords for Y axis
net = Network("600px", "700px", notebook=True) 
# disable physics  
net.toggle_physics(False) 
net.add_nodes(new_list_of_nodes, # use new reduced list of nodes instead of the old one 
       x=pos_x, # use new coordinates 
       y=pos_y)
# instead of adding edges from the list_of_edges, I am adding them straight from networkX graph object, because
# this way I do not need to calculate new list of edges 
net.add_edges(list(g.edges))
net.inherit_edge_colors(False)
net.set_edge_smooth('horizontal')

In [334]:
net.show('network_graph.html') # this looks much prettier 